In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

https://data.usaid.gov/Health/USAID-GHSC-PSM-Health-Commodity-Delivery-Dataset/tikn-bfy4/about_data

In [2]:
data = pd.read_excel("Supply chain logisitcs problem.xlsx", sheet_name="FreightRates")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540 entries, 0 to 1539
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Carrier       1540 non-null   object 
 1   orig_port_cd  1540 non-null   object 
 2   dest_port_cd  1540 non-null   object 
 3   minm_wgh_qty  1540 non-null   float64
 4   max_wgh_qty   1540 non-null   float64
 5   svc_cd        1540 non-null   object 
 6   minimum cost  1540 non-null   float64
 7   rate          1540 non-null   float64
 8   mode_dsc      1540 non-null   object 
 9   tpt_day_cnt   1540 non-null   int64  
 10  Carrier type  1540 non-null   object 
dtypes: float64(4), int64(1), object(6)
memory usage: 132.5+ KB


In [4]:
data.head()

,Carrier,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum cost,rate,mode_dsc,tpt_day_cnt,Carrier type
0,V444_6,PORT08,PORT09,250.0,499.99,DTD,43.2272,0.7132,AIR,2,V88888888_0
1,V444_6,PORT08,PORT09,65.0,69.99,DTD,43.2272,0.7512,AIR,2,V88888888_0
2,V444_6,PORT08,PORT09,60.0,64.99,DTD,43.2272,0.7892,AIR,2,V88888888_0
3,V444_6,PORT08,PORT09,50.0,54.99,DTD,43.2272,0.8272,AIR,2,V88888888_0
4,V444_6,PORT08,PORT09,35.0,39.99,DTD,43.2272,1.0552,AIR,2,V88888888_0


In [5]:
data = data.rename(
    columns={
    "Carrier": "carrier",
    'orig_port_cd': "origin_port_code",
    'dest_port_cd': "destination_port_code",
    'minm_wgh_qty': "min_weight_quantity",
    'max_wgh_qty': "max_weight_quantity", 
    'svc_cd': "service_code", 
    'mode_dsc': "mode_description",
    'tpt_day_cnt': "transport_day_count", 
    'Carrier type': "carrier_type"
    }
)

In [6]:
data.columns

Index(['carrier', 'origin_port_code', 'destination_port_code',
       'min_weight_quantity', 'max_weight_quantity', 'service_code',
       'minimum cost', 'rate', 'mode_description', 'transport_day_count',
       'carrier_type'],
      dtype='object')

Renamed columns to make them more understandable.

## Column Meaning

1.**carrier**: Represents the shipping or transport carrier company. Each carrier might be associated with specific services, transport modes, and costs.
	
2.**origin_port_code**: Stands for origin port code. This column represents the code or identifier for the location (port) from which the shipment originates.

3.**destination_port_code**: Stands for destination port code. This column represents the code or identifier for the location (port) where the shipment is delivered.

4.**minimun_weight_quantity**: Short for minimum weight quantity. This represents the minimum weight (in a specific unit) for which the rate or cost applies.

5.**max_weight_quantity** Short for maximum weight quantity. This represents the maximum weight (in a specific unit) for which the rate or cost applies.

6.**service_code**: Stands for service code. This represents the type or level of service offered by the carrier (e.g., dtp: door-to-port, dtd: door-to-door, etc.).

7.**minimum_cost** : Represents the minimum cost charged for the shipment within the specified weight range, regardless of the exact weight.

8.**rate**: Represents the cost rate (likely per unit of weight) that is charged by the carrier for the shipment. This value is applied based on the minm_wgh_qty and max_wgh_qty.

9.**mode_description**: Short for mode description. This represents the transportation mode used by the carrier (e.g., AIR, GROUND).

10.**transport_day_count**: Short for transport day count. This column represents the number of days it will take for the shipment to be transported from the origin port to the destination port.
	
11.**carrier_type**: Represents the type or classification of the carrier (e.g., based on the nature of the service, company type, or other differentiators). It could be a code or description for internal categorization.

## Checking for Duplicates or Missing Values

### Missing Values: 

In [9]:
data['rate'].describe()

count    1540.000000
mean        2.892656
std         4.603877
min         0.033200
25%         0.470400
50%         1.661200
75%         3.932200
max       128.027200
Name: rate, dtype: float64

In [7]:
data.isna().sum()

carrier                  0
origin_port_code         0
destination_port_code    0
min_weight_quantity      0
max_weight_quantity      0
service_code             0
minimum cost             0
rate                     0
mode_description         0
transport_day_count      0
carrier_type             0
dtype: int64

### Duplicate rows

In [8]:
print(data.duplicated().sum())

3


In [9]:
data[data.duplicated()]

,carrier,origin_port_code,destination_port_code,min_weight_quantity,max_weight_quantity,service_code,minimum cost,rate,mode_description,transport_day_count,carrier_type
1494,V444_5,PORT03,PORT09,0.0,5000.0,DTD,27.2024,9.2024,GROUND,1,V88888888_0
1505,V444_0,PORT03,PORT09,0.0,5000.0,DTD,27.2024,13.2024,GROUND,1,V88888888_0
1510,V444_0,PORT03,PORT09,0.0,5000.0,DTD,27.2024,13.2024,GROUND,1,V88888888_0


In [10]:
data = data.drop(index=1505)
data.reset_index(inplace=True, drop=True)

After analyzing the duplicates, we found three rows. The first row differs from the others due to a different value in the rate column. Therefore, we will drop either the second or the third row, as they are identical in all other columns.

## Feature Engineering

In [11]:
label_encoder = LabelEncoder()

In [14]:
categorical_columns = [
    'carrier',
    'origin_port_code',
    'destination_port_code',
    'mode_description',
    'service_code',
    'carrier_type'
]

for column in categorical_columns:
    label_encoder = LabelEncoder()
    data[column] = label_encoder.fit_transform(data[column])

In [18]:
data['rate']

0       0.7132
1       0.7512
2       0.7892
3       0.8272
4       1.0552
         ...  
1534    0.4116
1535    0.3112
1536    0.4116
1537    0.3112
1538    0.4116
Name: rate, Length: 1539, dtype: float64